In [333]:
import os
import copy
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow_decision_forests as tfdf

ModuleNotFoundError: No module named 'tensorflow_decision_forests'

In [2]:
files = os.listdir('./Fourrier/')
print(files)
files2 = []
for i in range(int(len(files)/10)):
    files2.append(files[i*10])
print(files2)
Viable_Cols = ['Acceleration x (m/s^2)_kalman', 'Acceleration y (m/s^2)_kalman', 'Acceleration z (m/s^2)_kalman', 'Gyroscope x (rad/s)_kalman',
              'Gyroscope y (rad/s)_kalman', 'Gyroscope z (rad/s)_kalman', 'Linear Acceleration x (m/s^2)_kalman',
              'Linear Acceleration y (m/s^2)_kalman', 'Linear Acceleration z (m/s^2)_kalman']
Viable_Cols2 = []
Viable_Cols3 = copy.deepcopy(Viable_Cols)
for column in Viable_Cols:
    newCol = str(column+'_max_freq')
    Viable_Cols2.append(newCol)
    Viable_Cols3.append(newCol)

['Fourrier Austin Biking 30rpm.csv', 'Fourrier Austin Biking 30rpm.csv 0.png', 'Fourrier Austin Biking 30rpm.csv 1.png', 'Fourrier Austin Biking 30rpm.csv 2.png', 'Fourrier Austin Biking 30rpm.csv 3.png', 'Fourrier Austin Biking 30rpm.csv 4.png', 'Fourrier Austin Biking 30rpm.csv 5.png', 'Fourrier Austin Biking 30rpm.csv 6.png', 'Fourrier Austin Biking 30rpm.csv 7.png', 'Fourrier Austin Biking 30rpm.csv 8.png', 'Fourrier Austin Biking 60rpm.csv', 'Fourrier Austin Biking 60rpm.csv 0.png', 'Fourrier Austin Biking 60rpm.csv 1.png', 'Fourrier Austin Biking 60rpm.csv 2.png', 'Fourrier Austin Biking 60rpm.csv 3.png', 'Fourrier Austin Biking 60rpm.csv 4.png', 'Fourrier Austin Biking 60rpm.csv 5.png', 'Fourrier Austin Biking 60rpm.csv 6.png', 'Fourrier Austin Biking 60rpm.csv 7.png', 'Fourrier Austin Biking 60rpm.csv 8.png', 'Fourrier Austin Biking 90rpm.csv', 'Fourrier Austin Biking 90rpm.csv 0.png', 'Fourrier Austin Biking 90rpm.csv 1.png', 'Fourrier Austin Biking 90rpm.csv 2.png', 'Fourrier

In [57]:
window = 200
slide = 0.1
train_test = 0.8
def make_windows(window, slide, frame, train_test):
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    for i in range(int(len(frame['Acceleration x (m/s^2)_kalman'])/(window*slide))-int((1/slide))):
        #print(i*20+200)
        sample = frame.loc[(i*20):(i*20)+199, Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*20)+199, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        if i*20 <= len(frame['Acceleration x (m/s^2)_kalman'])*train_test:
            train_samples.append(sample)
            sparse_train.append(sample2)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
        
        test_samples = test_samples[9:]
        sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test
frame = pd.read_csv('./Fourrier/Fourrier Austin Biking 60rpm.csv')
print(int(len(frame['Acceleration x (m/s^2)_kalman'])/(window*slide))-10)
print(len(frame['Acceleration x (m/s^2)']))
print(window*slide)
#train_samples, sparse_train, test_samples, sparse_test = make_windows(window, slide, frame, train_test)

279
5786
20.0


In [200]:
window = 200
slide = 0.1
train_test = 0.8
def make_windows_PCA(window, slide, frame, train_test):
    Viable_Cols = ['0', '1', '2', '3', '4', '5']
    Viable_Cols2 = ['0_max_freq', '1_max_freq', '2_max_freq', '3_max_freq', '4_max_freq', '5_max_freq', ]
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    labels_train = []
    labels_test = []
    if 'SPM/RPM' in frame.columns:
        label = 'SPM/RPM'
    if 'RPM/SPM' in frame.columns:
        label = 'RPM/SPM'
    if 'RPM' in frame.columns:
        label = 'RPM'
    if 'SPM' in frame.columns:
        label = 'SPM'
    for i in range(int(len(frame['0'])/(window*slide))-int((1/slide))):
        sample = frame.loc[(i*jump)+1:(i*jump)+(window), Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*jump)+200, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        means = [np.mean(sample[col]) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        sample2.append(means)
        yval = frame[label][(i*jump)+199]
        if i*jump <= (len(frame['0'])*train_test):
            train_samples.append(sample)
            sparse_train.append(sample2)
            labels_train.append(yval)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
            labels_test.append(yval)
    labels_test = labels_test[9:]
    test_samples = test_samples[9:]
    sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test

In [ ]:
files = os.listdir('./PCA Fourier/')
bike_train_samples, bike_sparse_train, bike_test_samples, bike_sparse_test, bike_labels_train, bike_labels_test = [],[],[],[],[],[]
walk_train_samples, walk_sparse_train, walk_test_samples, walk_sparse_test, walk_labels_train, walk_labels_test = [],[],[],[],[],[]
run_train_samples, run_sparse_train, run_test_samples, run_sparse_test, run_labels_train, run_labels_test = [],[],[],[],[],[]

Biking_Files = [0,1,2,16,17]
for i in Biking_Files:
    print('A')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    bike_train_samples.extend(train_samples)
    bike_sparse_train.extend(sparse_train)
    bike_test_samples.extend(test_samples)
    bike_sparse_test.extend(sparse_test)
    bike_labels_train.extend(labels_train)
    bike_labels_test.extend(labels_test)
Walking_Files = [10,11,12,13,14,18,20,21]
for i in Walking_Files:
    print('B')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    walk_train_samples.extend(train_samples)
    walk_sparse_train.extend(sparse_train)
    walk_test_samples.extend(test_samples)
    walk_sparse_test.extend(sparse_test)
    walk_labels_train.extend(labels_train)
    walk_labels_test.extend(labels_test)
Running_Files = [4,5,6,7]
for i in Running_Files:
    print('C')
    frame = pd.read_csv('./PCA Fourier/'+str(files[i]))
    train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_PCA(window, slide, frame, train_test)
    run_train_samples.extend(train_samples)
    run_sparse_train.extend(sparse_train)
    run_test_samples.extend(test_samples)
    run_sparse_test.extend(sparse_test)
    run_labels_train.extend(labels_train)
    run_labels_test.extend(labels_test)


    

In [ ]:
print(len(run_train_samples))
print(len(walk_train_samples))
print(len(bike_train_samples))
print(len(run_test_samples))
print(len(walk_test_samples))
print(len(bike_test_samples))
print(len(bike_labels_test))
print(run_train_samples[1])
print(run_sparse_train[1])

def reshaper(frame):
    data_2d = frame.reshape(-1, frame.shape[-1])
    scaler = StandardScaler()
    whitened_data_2d = scaler.fit_transform(data_2d)
    whitened_frame = whitened_data_2d.reshape(frame.shape)
    return whitened_frame


In [ ]:
tf_run_sparse_train = np.array(run_sparse_train)
tf_run_labels_train = np.array(run_labels_train)
tf_run_sparse_test = np.array(run_sparse_test)
tf_run_labels_test = np.array(run_labels_test)

whitened_run_sparse_train = reshaper(tf_run_sparse_train)
whitened_run_sparse_test = reshaper(tf_run_sparse_test)

model_run = Sequential([
    Flatten(input_shape=(5,6)),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='relu')])
model_run.compile(optimizer=Adam(learning_rate=0.02), loss='mean_absolute_error', metrics=['mean_absolute_error'])
model_run.fit(whitened_run_sparse_train, tf_run_labels_train, epochs=10)
model_run.evaluate(whitened_run_sparse_test, tf_run_labels_test)


In [ ]:
tf_walk_sparse_train = np.array(walk_sparse_train)
tf_walk_labels_train = np.array(walk_labels_train)
tf_walk_sparse_test = np.array(walk_sparse_test)
tf_walk_labels_test = np.array(walk_labels_test)

whitened_walk_sparse_train = reshaper(tf_walk_sparse_train)
whitened_walk_sparse_test = reshaper(tf_walk_sparse_test)

model_walk = Sequential([
    Flatten(input_shape=(5,6)),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1, activation='relu')])
model_walk.compile(optimizer=Adam(learning_rate=0.1), loss='mean_absolute_error', metrics=['mean_absolute_error'])
model_walk.fit(whitened_walk_sparse_train, tf_walk_labels_train, epochs=10)
model_walk.evaluate(whitened_walk_sparse_test, tf_walk_labels_test)

In [311]:
tf_bike_sparse_train = np.array(bike_sparse_train)
tf_bike_labels_train = np.array(bike_labels_train)
tf_bike_sparse_test = np.array(bike_sparse_test)
tf_bike_labels_test = np.array(bike_labels_test)

whitened_bike_sparse_train = reshaper(tf_bike_sparse_train)
whitened_bike_sparse_test = reshaper(tf_bike_sparse_test)

model_bike = Sequential([
    Flatten(input_shape=(5,6)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='relu')])
model_bike.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error', metrics=['mean_absolute_error'])
model_bike.fit(whitened_bike_sparse_train, tf_bike_labels_train, epochs=10)
model_bike.evaluate(whitened_bike_sparse_test, tf_bike_labels_test)

Epoch 1/10
29/29 [==============================] - 0s 1ms/step - loss: 30.6762 - mean_absolute_error: 30.6762
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 16.6507 - mean_absolute_error: 16.6507
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 13.8011 - mean_absolute_error: 13.8011
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 11.0550 - mean_absolute_error: 11.0550
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 9.9959 - mean_absolute_error: 9.9959
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 11.7806 - mean_absolute_error: 11.7806
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 10.4581 - mean_absolute_error: 10.4581
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 11.3227 - mean_absolute_error: 11.3227
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 10.8202 - mean_absolute_error: 10.8202
Epo

[10.021334648132324, 10.021334648132324]

In [166]:
def make_windows_full_dataset(window, slide, frame, train_test):
    Viable_Cols = ['0', '1', '2', '3', '4', '5']
    Viable_Cols2 = ['0_max_freq', '1_max_freq', '2_max_freq', '3_max_freq', '4_max_freq', '5_max_freq', ]
    jump = window*slide
    train_samples = []
    sparse_train = []
    test_samples = []
    sparse_test = []
    labels_train = []
    labels_test = []
    for i in range(int(len(frame['0'])/(window*slide))-int((1/slide))):
        print(i*jump)
        sample = frame.loc[(i*jump):(i*jump)+(window-1), Viable_Cols]
        sample2 = []
        freqs = frame.loc[(i*jump)+199, Viable_Cols2]
        sample2.append(freqs.tolist())
        mins = [np.min(sample[col]) for col in Viable_Cols]
        maxes = [np.max(sample[col]) for col in Viable_Cols]
        stdev = [np.std(sample[col], axis=0) for col in Viable_Cols]
        means = [np.mean(sample[col]) for col in Viable_Cols]
        sample2.append(mins)
        sample2.append(maxes)
        sample2.append(stdev)
        sample2.append(means)
        if frame['Still'][(i*jump)+199] == 1:
            yval = 0
        if frame['Walk'][(i*jump)+199] == 1:
            yval = 1
        if frame['Run'][(i*jump)+199] == 1:
            yval = 2
        if frame['Cycle'][(i*jump)+199] == 1:
            yval = 3
        
        if i*jump <= (len(frame['0'])*train_test):
            train_samples.append(sample)
            sparse_train.append(sample2)
            labels_train.append(yval)
        else:
            test_samples.append(sample)
            sparse_test.append(sample2)
            labels_test.append(yval)
    labels_test = labels_test[9:]
    test_samples = test_samples[9:]
    sparse_test = sparse_test[9:]
    return train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test

def sampler(train_samples, sparse_train, labels_train, num):
    length = len(train_samples)
    ids = random.sample(range(length), num)
    dataSamples = []
    sparseSamples = []
    labelSamples = []
    for i in ids:
        dataSamples.append(train_samples[i])
        sparseSamples.append(sparse_train[i])
        labelSamples.append(labels_train[i])
    return dataSamples, sparseSamples, labelSamples

In [ ]:
frame = pd.read_csv('Complete_dataset_PCA_Fourier.csv')
train_samples, sparse_train, test_samples, sparse_test, labels_train, labels_test = make_windows_full_dataset(window, slide, frame, train_test)
num = 20
dataSamples, sparseSamples, labelSamples = sampler(train_samples, sparse_train, labels_train, num)
tf_sparse_train = np.array(sparse_train)
tf_labels_train = np.array(labels_train)
tf_sparse_test = np.array(sparse_test)
tf_labels_test = np.array(labels_test)
    


In [281]:
import tensorflow as tf
print(tf_sparse_train.shape)
whitened_tf_sparse_train = reshaper(tf_sparse_train[1:])
print(whitened_tf_sparse_train[0])

model_classify = tf.keras.models.Sequential([
    Flatten(input_shape=(5,6)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(4, activation='softmax')])
model_classify.compile(optimizer=tf.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_classify.fit(whitened_tf_sparse_train, tf_labels_train[1:], epochs=5)
whitened_tf_sparse_test = reshaper(tf_sparse_test[1:])
model_classify.evaluate(whitened_tf_sparse_test, tf_labels_test[1:])


(5792, 5, 6)
[[ 0.28095727 -0.16922378  0.98886819  1.03413009  0.50321319  0.04874273]
 [-0.53132363 -1.35084082 -1.02818922 -0.99882539 -0.54503232 -1.45949958]
 [ 0.38855584 -0.29629351  0.82851656  0.3124203   0.68795766  0.21776652]
 [-0.10627362 -0.11335044 -0.04700736 -0.05525641  0.00859163  0.03064378]
 [-0.1428993  -0.68177014 -0.30601845 -0.20604174 -0.048618   -0.77097134]]
Epoch 1/5
181/181 [==============================] - 1s 1ms/step - loss: 0.4162 - accuracy: 0.8499
Epoch 2/5
181/181 [==============================] - 0s 1ms/step - loss: 0.1844 - accuracy: 0.9249
Epoch 3/5
181/181 [==============================] - 0s 1ms/step - loss: 0.1528 - accuracy: 0.9289
Epoch 4/5
181/181 [==============================] - 0s 1ms/step - loss: 0.1384 - accuracy: 0.9346
Epoch 5/5
45/45 [==============================] - 0s 680us/step - loss: 0.5863 - accuracy: 0.8219


[0.5863395929336548, 0.8218793869018555]

In [331]:
predictions = model_classify.predict(whitened_tf_sparse_train)
still = []
walk = []
run = []
bike = []
for i, item in enumerate(predictions):
    if np.argmax(item) == 0:
        still.append(whitened_tf_sparse_train[i])
    if np.argmax(item) == 1:
        walk.append(whitened_tf_sparse_train[i])
    if np.argmax(item) == 2:
        run.append(whitened_tf_sparse_train[i])
    if np.argmax(item) == 3:
        bike.append(whitened_tf_sparse_train[i])
walk_cad = model_walk.predict(np.array(walk))
run_cad = model_run.predict(np.array(run))
bike_cad = model_bike.predict(np.array(bike))
walk_time = len(walk)/(60)
walk_avg = np.mean(walk_cad)
run_time = len(run)/(60)
run_avg = np.mean(run_cad)
bike_time = len(bike)/(60)
bike_avg = np.mean(bike_cad)
print("You walked for "+str(round(walk_time,2))+" minutes at a pace of "+str(int(walk_avg))+" steps per minute on average")
print("You ran for "+str(round(run_time,2))+" minutes at a pace of "+str(int(run_avg))+" steps per minute on average")
print("You biked for "+str(round(bike_time,2))+" minutes at a pace of "+str(int(bike_avg))+" revolutions per minute on average")



31/31 [==============================] - 0s 574us/step
You walked for 45.9 minutes at a pace of 72 steps per minute on average
You ran for 15.85 minutes at a pace of 148 steps per minute on average
You biked for 16.28 minutes at a pace of 48 revolutions per minute on average


In [345]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
randomForest = RandomForestClassifier()
whitened_tf_sparse_train_flat = whitened_tf_sparse_train.reshape(whitened_tf_sparse_train.shape[0], -1)
whitened_tf_sparse_test_flat = whitened_tf_sparse_test.reshape(whitened_tf_sparse_test.shape[0], -1)
randomForest.fit(whitened_tf_sparse_train_flat, tf_labels_train[1:])
train_predictions = randomForest.predict(whitened_tf_sparse_train_flat)
predictions = randomForest.predict(whitened_tf_sparse_test_flat)
accuracy = accuracy_score(tf_labels_test[1:], predictions)
print(f'Test accuracy: {accuracy}')
confusion = confusion_matrix(tf_labels_test[1:], predictions)
print(confusion)
confusion_train = confusion_matrix(tf_labels_train[1:], train_predictions)
print(confusion_train)


Test accuracy: 0.9144460028050491
[[  1   0   0   0]
 [  0 531   5   0]
 [  2  61 232   0]
 [ 45   9   0 540]]
[[1072    0    0    0]
 [   0 2958    0    0]
 [   0    0  943    0]
 [   0    0    0  818]]
